In [ ]:
pip install torch torchaudio transformers datasets peft bitsandbytes accelerate torchcodec dataset evaluate bitsandbytes jiwer -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
google-adk 1.19.0 requires sqlalchemy<3.0.0,>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
HF_TOKEN="xxx"  # --- Add Hugging Face Token ---

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model_name = "openai/whisper-small.en"
processor = WhisperProcessor.from_pretrained(model_name)  # (same as original, reused everywhere)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training  # (original import)

# CHANGED: instead of creating a single global model once,
# we wrap the original QLoRA setup into a helper so we can
# create a fresh model *per dataset* (regular vs trimmed).
def create_lora_model():
    """Create a fresh 4-bit QLoRA Whisper model (same config as original)."""
    model = WhisperForConditionalGeneration.from_pretrained(
        model_name,
        load_in_4bit=True,               # quantize base model
        device_map="auto",
    )

    # For QLoRA (same as original)
    model = prepare_model_for_kbit_training(model)  # enables 4-bit quantization

    config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],  # or attention layers depending on model
        lora_dropout=0.1,
        bias="none",
        task_type="SEQ_2_SEQ_LM"  # or "CTC" for wav2vec2
    )

    model = get_peft_model(model, config)
    model.print_trainable_parameters()

    # CHANGED: the forward patch originally lived at global scope;
    # now we apply it inside this helper so each new model is patched.
    from types import MethodType

    def patched_forward(self, input_features=None, **kwargs):
        return self.model.forward(input_features=input_features, **kwargs)

    model.forward = MethodType(patched_forward, model)  # CHANGED: moved from global into helper

    return model  # NEW


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# from datasets import load_dataset
from google.colab import drive
import os

drive.mount('/content/drive')

import os, json
from datasets import Dataset, Audio

# ---- CONFIG ----
SAMPLING_RATE = 16_000  # set based on your ASR model

# NEW: two dataset directories instead of one, as per your requirement.
DATA_DIR_REGULAR = '/content/drive/My Drive/VoiceBridge_SAP_sample'          # CHANGED: was VoiceBridge_SAP_sample
DATA_DIR_TRIMMED = '/content/drive/My Drive/VoiceBridge_SAP_sample_trimmed'  # NEW
META_FILENAME = "digital_assistant_metadata.json"  # NEW: common metadata filename


# NEW: factor out the original metadata + dataset-building logic so we can reuse it.
def build_dataset_from_dir(data_dir: str):
    """
    Load metadata and build a HuggingFace Dataset from a given directory.
    This is the same logic as your original code, but parameterized by `data_dir`.
    """
    meta_file = os.path.join(data_dir, META_FILENAME)  # CHANGED: use parameterized dir

    # ---- LOAD METADATA ----
    with open(meta_file, "r") as f:
        metadata = json.load(f)

    # ---- BUILD LIST OF AUDIO–TEXT PAIRS ----
    data = []
    for item in metadata:
        filename = item.get("Filename")
        transcript = item.get("Prompt", {}).get("Transcript", "")
        if not filename or not transcript:
            continue
        audio_path = os.path.join(data_dir, filename)  # CHANGED: was DATA_DIR
        if os.path.exists(audio_path):
            data.append({"audio": audio_path, "sentence": transcript})

    print(f"[{data_dir}] Found {len(data)} usable audio–transcript pairs")

    # ---- CREATE HUGGING FACE DATASET ----
    dataset = Dataset.from_list(data)
    # dataset = dataset.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))
    dataset = dataset.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE, decode=False))
    return dataset  # NEW

Mounted at /content/drive


In [ ]:
# (kept from original)
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small.en")

In [ ]:
import torchaudio


def preprocess(batch):
    speech_arrays = []
    for audio_info in batch["audio"]:
        path = audio_info["path"]
        speech_array, sr = torchaudio.load(path)

        # CHANGED: always downmix to mono so every sample is 1D
        if speech_array.ndim == 2:               # (channels, samples)
            # average across channels -> mono
            speech_array = speech_array.mean(dim=0)

        # CHANGED: ensure we end up with a 1D numpy array
        speech_arrays.append(speech_array.numpy())

    # Feature extraction: do NOT use return_tensors here
    input_features = processor.feature_extractor(
        speech_arrays,
        sampling_rate=SAMPLING_RATE,
    ).input_features  # list of arrays

    # Tokenize transcripts (unchanged)
    labels = processor.tokenizer(
        batch["sentence"],
        add_special_tokens=True
    ).input_ids

    return {"input_features": input_features, "labels": labels}



In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        labels_batch = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels = self.processor.tokenizer.pad(labels_batch, return_tensors="pt")["input_ids"]

        batch["labels"] = labels.masked_fill(labels == self.processor.tokenizer.pad_token_id, -100)
        return batch


data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import numpy as np
import evaluate

wer_metric = evaluate.load("wer")


def compute_metrics(pred):
    pred_ids = np.argmax(pred.predictions, axis=-1)
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(pred.label_ids, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}


In [ ]:
from transformers import TrainingArguments, Trainer


# NEW: wrapper to do the *entire* training pipeline for a given folder.
def train_q_lora_for_folder(data_dir: str, output_dir_root: str):
    """
    Train a QLoRA Whisper model on the dataset in `data_dir`.
    This wraps your original "dataset + trainer + train + save" code so
    we can call it for both regular and trimmed folders.
    """
    print(f"\n=== Training QLoRA model for: {data_dir} ===")

    # 1) Build dataset (original logic, now via helper)
    dataset = build_dataset_from_dir(data_dir)  # NEW

    # 2) Map preprocess (same as original)
    dataset = dataset.map(
        preprocess,
        # remove_columns=["audio", "sentence"],
        batched=True,
        batch_size=4,
        num_proc=1,
    )

    # 3) Split train / eval (same as original)
    dataset = dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = dataset["train"]
    eval_dataset = dataset["test"]

    # 4) Create a fresh LoRA model (NEW vs original single global model)
    model = create_lora_model()  # NEW

    # 5) Optional: quick forward pass check (same logic as original)
    batch = data_collator([train_dataset[0], train_dataset[1]])
    batch = {k: v.to(model.device) for k, v in batch.items()}

    outputs = model(
        input_features=batch["input_features"],
        labels=batch["labels"]
    )
    print("Sanity check loss:", outputs.loss.item())

    # 6) Training arguments (same as original, but output_dir is parameterized)
    training_args = TrainingArguments(
        output_dir=output_dir_root,          # CHANGED: was "./whisper-lora-checkpoints"
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        learning_rate=1e-4,
        num_train_epochs=3,
        fp16=False,
        bf16=False,
        logging_strategy="steps",
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        report_to="none",
        load_best_model_at_end=True,
    )

    # 7) Trainer (same as original)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=None,
        compute_metrics=compute_metrics,
    )

    # 8) Train (same as original)
    trainer.train()

    # 9) Save adapters and merged model (same logic, but under output_dir_root)
    adapter_dir = os.path.join(output_dir_root, "whisper-lora-adapter")   # NEW
    merged_dir = os.path.join(output_dir_root, "whisper-lora-merged")     # NEW

    model.save_pretrained(adapter_dir)

    merged_model = model.merge_and_unload()
    merged_model.save_pretrained(merged_dir)

    processor.save_pretrained(merged_dir)

    print(f"Saved adapter to: {adapter_dir}")
    print(f"Saved merged model to: {merged_dir}")

    # Return merged_model so we can evaluate WER on it
    return merged_model, merged_dir, eval_dataset  # NEW


In [ ]:
OUTPUT_DIR_REGULAR = "/content/drive/My Drive/whisper-lora-regular-checkpoints"

model_regular, merged_dir_regular, eval_dataset_regular = train_q_lora_for_folder(
    DATA_DIR_REGULAR,
    OUTPUT_DIR_REGULAR,
)




=== Training QLoRA model for: /content/drive/My Drive/VoiceBridge_SAP_sample ===
[/content/drive/My Drive/VoiceBridge_SAP_sample] Found 207 usable audio–transcript pairs


Map:   0%|          | 0/207 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 1,769,472 || all params: 243,503,616 || trainable%: 0.7267


You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Sanity check loss: 7.226772785186768


/tmp/ipython-input-506667662.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the i

Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 357, 366, 438, 532, 685, 705, 796, 930, 1058, 1220, 1267, 1279, 1303, 1343, 1377, 1391, 1635, 1782, 1875, 2162, 2361, 2488, 3467, 4008, 4211, 4600, 4808, 5299, 5855, 6329, 7203, 9609, 9959, 10563, 10786, 11420, 11709, 11907, 13163, 13697, 13700, 14808, 15306, 16410, 16791, 17992, 19203, 19510, 20724, 22305, 22935, 27007, 30109, 30420, 33409, 34949, 40283, 40493, 40549, 47282, 49146, 50257, 50357, 50358, 50359, 50360, 50361]}. You are seeing this warning because you've set generation parameters in the model config, as o

Saved adapter to: /content/drive/My Drive/whisper-lora-regular-checkpoints/whisper-lora-adapter
Saved merged model to: /content/drive/My Drive/whisper-lora-regular-checkpoints/whisper-lora-merged


In [ ]:
OUTPUT_DIR_TRIMMED = "/content/drive/My Drive/whisper-lora-trimmed-checkpoints"

model_trimmed, merged_dir_trimmed, eval_dataset_trimmed = train_q_lora_for_folder(
    DATA_DIR_TRIMMED,
    OUTPUT_DIR_TRIMMED,
)


=== Training QLoRA model for: /content/drive/My Drive/VoiceBridge_SAP_sample_trimmed ===
[/content/drive/My Drive/VoiceBridge_SAP_sample_trimmed] Found 207 usable audio–transcript pairs


Map:   0%|          | 0/207 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 1,769,472 || all params: 243,503,616 || trainable%: 0.7267
Sanity check loss: 7.507758617401123


/tmp/ipython-input-506667662.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the i

Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 357, 366, 438, 532, 685, 705, 796, 930, 1058, 1220, 1267, 1279, 1303, 1343, 1377, 1391, 1635, 1782, 1875, 2162, 2361, 2488, 3467, 4008, 4211, 4600, 4808, 5299, 5855, 6329, 7203, 9609, 9959, 10563, 10786, 11420, 11709, 11907, 13163, 13697, 13700, 14808, 15306, 16410, 16791, 17992, 19203, 19510, 20724, 22305, 22935, 27007, 30109, 30420, 33409, 34949, 40283, 40493, 40549, 47282, 49146, 50257, 50357, 50358, 50359, 50360, 50361]}. You are seeing this warning because you've set generation parameters in the model config, as o

Saved adapter to: /content/drive/My Drive/whisper-lora-trimmed-checkpoints/whisper-lora-adapter
Saved merged model to: /content/drive/My Drive/whisper-lora-trimmed-checkpoints/whisper-lora-merged


In [ ]:
!pip install -q evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 5.6 MB/s eta 0:00:00


In [ ]:
pip install -U bitsandbytes

In [ ]:
# =========================
# Merge LoRA checkpoints and test the models
# =========================

import os
import json
import soundfile as sf
import librosa
import pandas as pd
from tqdm import tqdm
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import evaluate

# --- paths (must match what you used in training) ---
DATA_DIR_REGULAR = "/content/drive/My Drive/VoiceBridge_SAP_sample"
DATA_DIR_TRIMMED = "/content/drive/My Drive/VoiceBridge_SAP_sample_trimmed"

# These are the *parent* dirs where training outputs live
OUTPUT_DIR_REGULAR = "/content/drive/My Drive/whisper-lora-regular-checkpoints"
OUTPUT_DIR_TRIMMED = "/content/drive/My Drive/whisper-lora-trimmed-checkpoints"

# These are the LoRA adapter dirs (where adapter_config.json actually is)
ADAPTER_DIR_REGULAR = os.path.join(OUTPUT_DIR_REGULAR, "whisper-lora-adapter")
ADAPTER_DIR_TRIMMED = os.path.join(OUTPUT_DIR_TRIMMED, "whisper-lora-adapter")

# Where we'll save merged full models
MERGED_DIR_REGULAR = os.path.join(OUTPUT_DIR_REGULAR, "whisper-lora-merged")
MERGED_DIR_TRIMMED = os.path.join(OUTPUT_DIR_TRIMMED, "whisper-lora-merged")

BASE_MODEL_ID = "openai/whisper-small.en"

# ----------------------------------------------------
# 1) Merge LoRA adapters into full Whisper models
# ----------------------------------------------------

processor_base = WhisperProcessor.from_pretrained(BASE_MODEL_ID)

def merge_lora_checkpoint(adapter_dir: str, merged_dir: str):
    """
    Load base Whisper, attach LoRA from `adapter_dir`, merge, and save
    a normal (non-quantized) model into `merged_dir`.
    """
    print(f"\n[MERGE] From adapter dir: {adapter_dir}")
    print(f"[MERGE] To merged dir:    {merged_dir}")

    if not os.path.isdir(adapter_dir):
        raise FileNotFoundError(f"Adapter dir not found: {adapter_dir}")

    # Load base model in full precision (no bitsandbytes)
    base_model = WhisperForConditionalGeneration.from_pretrained(
        BASE_MODEL_ID,
        torch_dtype=torch.float32,
    )

    # Attach LoRA weights from adapter directory
    lora_model = PeftModel.from_pretrained(base_model, adapter_dir)

    # Merge LoRA into the base weights
    lora_model = lora_model.merge_and_unload()

    # Save merged model + processor
    os.makedirs(merged_dir, exist_ok=True)
    lora_model.to("cpu")
    lora_model.save_pretrained(merged_dir)
    processor_base.save_pretrained(merged_dir)

    print("[MERGE] Done.")

# Merge both REGULAR and TRIMMED LoRA checkpoints
merge_lora_checkpoint(ADAPTER_DIR_REGULAR, MERGED_DIR_REGULAR)
merge_lora_checkpoint(ADAPTER_DIR_TRIMMED, MERGED_DIR_TRIMMED)

print("\n[MERGE] Finished merging both models.\n")

# ----------------------------------------------------
# 2) Load merged models (no quantization, no bitsandbytes)
# ----------------------------------------------------

processor = WhisperProcessor.from_pretrained(BASE_MODEL_ID)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model_regular = WhisperForConditionalGeneration.from_pretrained(
    MERGED_DIR_REGULAR,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)
model_regular.to(device)

model_trimmed = WhisperForConditionalGeneration.from_pretrained(
    MERGED_DIR_TRIMMED,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)
model_trimmed.to(device)

baseline_model = WhisperForConditionalGeneration.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)
baseline_model.to(device)

# ----------------------------------------------------
# 3) WER function for Whisper (nested folder layout)
# ----------------------------------------------------

def transcribe_whisper_and_compute_wer(
    root_folder,
    model,
    device=None,
    batch_size=4,
):
    """
    Walk `root_folder`, find all .json files, build:
        abs wav path -> transcript
    Then transcribe all wavs and compute WER per file.
    """
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    # 1) Build reference dict
    reference_dict = {}
    for root, _, files in os.walk(root_folder):
        for fname in files:
            if fname.lower().endswith(".json"):
                json_path = os.path.join(root, fname)
                with open(json_path, "r") as f:
                    metadata = json.load(f)

                for entry in metadata:
                    wav_name = entry.get("Filename")
                    transcript = entry.get("Prompt", {}).get("Transcript", "")
                    if not wav_name or not transcript:
                        continue
                    wav_path = os.path.join(root, wav_name)
                    reference_dict[wav_path] = transcript.strip().lower()

    print(f"[{root_folder}] Found {len(reference_dict)} reference entries in JSON files")

    # 2) Collect all wav files
    wav_files = sorted(
        os.path.join(root, f)
        for root, _, files in os.walk(root_folder)
        for f in files
        if f.lower().endswith(".wav")
    )
    print(f"[{root_folder}] Found {len(wav_files)} wav files")

    # 3) put model on device
    model.to(device)
    model.eval()

    metric = evaluate.load("wer")

    def load_audio(path, target_sr=16000):
        speech_array, sr = sf.read(path)
        if sr != target_sr:
            speech_array = librosa.resample(speech_array, orig_sr=sr, target_sr=target_sr)
        return speech_array

    @torch.inference_mode()
    def batch_transcribe(wav_paths):
        transcriptions = []
        for i in tqdm(
            range(0, len(wav_paths), batch_size),
            desc=f"Transcribing ({os.path.basename(root_folder)})"
        ):
            batch = [load_audio(p) for p in wav_paths[i:i+batch_size]]

            # Encode audio -> input_features + attention_mask
            encoded = processor(
                batch,
                sampling_rate=16000,
                return_tensors="pt",
                padding=True,
            )

            input_features = encoded.input_features.to(
                device=device,
                dtype=model.dtype,   # <-- match model (float16 on GPU, float32 on CPU)
            )

            attention_mask = getattr(encoded, "attention_mask", None)
            if attention_mask is not None:
                attention_mask = attention_mask.to(device)

                pred_ids = model.generate(
                    input_features=input_features,
                    attention_mask=attention_mask,
                )
            else:
                pred_ids = model.generate(
                    input_features=input_features,
                )

            decoded = processor.batch_decode(pred_ids, skip_special_tokens=True)
            transcriptions.extend(t.strip().lower() for t in decoded)

        return transcriptions


    preds = batch_transcribe(wav_files)

    # 4) compute WER per file
    results = []
    for wav_path, pred in zip(wav_files, preds):
        ref = reference_dict.get(wav_path, "")
        wer = metric.compute(predictions=[pred], references=[ref])
        results.append({
            "Path": wav_path,
            "Filename": os.path.basename(wav_path),
            "Prediction": pred,
            "Reference": ref,
            "WER": wer,
        })

    results_df = pd.DataFrame(results)

    overall_wer = results_df["WER"].mean()
    print(f"\n[{os.path.basename(root_folder)}] Overall average WER: {overall_wer:.3f}")

    top10 = results_df.sort_values("WER", ascending=False).head(10)
    print("\nTop 10 highest WER samples:")
    print(top10[["Filename", "WER", "Reference", "Prediction"]].to_string(index=False))

    return results_df

# ----------------------------------------------------
# 4) Run evaluations
# ----------------------------------------------------

results_regular_df = transcribe_whisper_and_compute_wer(
    DATA_DIR_REGULAR,
    model=model_regular,
    device=device,
)

results_trimmed_df = transcribe_whisper_and_compute_wer(
    DATA_DIR_TRIMMED,
    model=model_trimmed,
    device=device,
)

baseline_results_regular_df = transcribe_whisper_and_compute_wer(
    DATA_DIR_REGULAR,
    model=baseline_model,
    device=device,
)



[MERGE] From adapter dir: /content/drive/My Drive/whisper-lora-regular-checkpoints/whisper-lora-adapter
[MERGE] To merged dir:    /content/drive/My Drive/whisper-lora-regular-checkpoints/whisper-lora-merged


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 357, 366, 438, 532, 685, 705, 796, 930, 1058, 1220, 1267, 1279, 1303, 1343, 1377, 1391, 1635, 1782, 1875, 2162, 2361, 2488, 3467, 4008, 4211, 4600, 4808, 5299, 5855, 6329, 7203, 9609, 9959, 10563, 10786, 11420, 11709, 11907, 13163, 13697, 13700, 14808, 15306, 16410, 16791, 17992, 19203, 19510, 20724, 22305, 22935, 27007, 30109, 30420, 33409, 34949, 40283, 40493, 40549, 47282, 49146, 50257, 50357, 50358, 50359, 50360, 50361]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


[MERGE] Done.

[MERGE] From adapter dir: /content/drive/My Drive/whisper-lora-trimmed-checkpoints/whisper-lora-adapter
[MERGE] To merged dir:    /content/drive/My Drive/whisper-lora-trimmed-checkpoints/whisper-lora-merged
[MERGE] Done.

[MERGE] Finished merging both models.

Using device: cuda
[/content/drive/My Drive/VoiceBridge_SAP_sample] Found 7431 reference entries in JSON files
[/content/drive/My Drive/VoiceBridge_SAP_sample] Found 207 wav files


Transcribing (VoiceBridge_SAP_sample):   0%|          | 0/52 [00:00<?, ?it/s]A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsPr


[VoiceBridge_SAP_sample] Overall average WER: 0.757

Top 10 highest WER samples:
                                            Filename  WER          Reference                       Prediction
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29119_13128.wav  5.0      escitalopram.       and please stay long. bye.
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29338_13367.wav  4.0           avocado.                  i will call on.
23b27207-6d7f-4da9-a85b-08dcf77c2ab9_29853_13492.wav  4.0         audiobook.               how will you book?
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29122_13127.wav  3.0      take focalin.  they thought they would use it.
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29111_12390.wav  3.0           aricept.              bye, little one-fa.
  2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_148_13127.wav  2.0           cortana.                        good job.
  2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_151_12390.wav  2.0            redial.                      with the...
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_2

Transcribing (VoiceBridge_SAP_sample_trimmed): 100%|██████████| 52/52 [02:14<00:00,  2.59s/it]



[VoiceBridge_SAP_sample_trimmed] Overall average WER: 0.819

Top 10 highest WER samples:
                                            Filename  WER        Reference            Prediction
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29119_13128.wav  5.0    escitalopram. at the state of life.
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29338_13367.wav  4.0         avocado.      i will carry on.
    0c3a355b-c4da-40fe-57de-08dcbb9f64e1_29_8160.wav  2.0         cortana.          from talent.
23b27207-6d7f-4da9-a85b-08dcf77c2ab9_29853_13492.wav  2.0       audiobook.             aria book
23b27207-6d7f-4da9-a85b-08dcf77c2ab9_29349_13492.wav  2.0        arriving.              bye bye.
  2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_148_13127.wav  2.0         cortana.           with color.
  28fe23ad-6c0d-411b-03f3-08dd318df37f_148_13474.wav  2.0         cortana.            good time.
  f6616ff1-176f-477d-1cda-08dce2a2fe9b_151_13018.wav  2.0          redial.              read out
  2dcb3ebc-59ea-4275-75a6-08dd3ba78fc

Transcribing (VoiceBridge_SAP_sample): 100%|██████████| 52/52 [00:09<00:00,  5.78it/s]



[VoiceBridge_SAP_sample] Overall average WER: 0.779

Top 10 highest WER samples:
                                            Filename      WER              Reference                       Prediction
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29119_13128.wav 6.000000          escitalopram.    and please stay in our place.
23b27207-6d7f-4da9-a85b-08dcf77c2ab9_29853_13492.wav 4.000000             audiobook.              how we have worked?
23b27207-6d7f-4da9-a85b-08dcf77c2ab9_29108_13492.wav 3.000000              adderall?                     up the ball.
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29122_13127.wav 3.000000          take focalin.  they thought they would use it.
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29338_13367.wav 3.000000               avocado.                    i will cover.
2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_29111_12390.wav 2.000000               aricept.                     bye, loonfa.
  2dcb3ebc-59ea-4275-75a6-08dd3ba78fcd_148_13127.wav 2.000000               cortana.        